## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ax.plot.trace import optimization_trace_single_method

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [4]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [5]:
Tag_client = AxClient()

[INFO 04-05 13:44:57] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [6]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [5, 15], #NOTE - z_sh is actually half z_s (which must be even)
    "z_p2": [25, 35],
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.5, 0.5],
    "x_p1": [-1.0, 1.0],
    "x_p2": [-0.5, 0.5],
    "x_r2": [-1.5, 1.5],
}

# Number of copies for each parameter
num_copies = 3

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.01*(num_copies-1-i)), upper - .01*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        # parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Viable_Cross_Terms": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-05 13:44:57] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[7, 15]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[6, 14]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[5, 13]), RangeParameter(name='z_p2', parameter_type=INT, range=[27, 35]), RangeParameter(name='z_p2_1', parameter_type=INT, range=[26, 34]), RangeParameter(name='z_p2_2', parameter_type=INT, range=[25, 33]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.48, 0.5]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.49, 0.49]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.5, 0.48]), RangeParameter(name='x_p1', parameter_type=FLOAT, range=[-0.98, 1.0]), RangeParameter(name=

In [7]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [8]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [7, 15], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [6, 14], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [5, 13], 'value_type': 'int'},
 {'name': 'z_p2', 'type': 'range', 'bounds': [27, 35], 'value_type': 'int'},
 {'name': 'z_p2_1', 'type': 'range', 'bounds': [26, 34], 'value_type': 'int'},
 {'name': 'z_p2_2', 'type': 'range', 'bounds': [25, 33], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [72, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [71, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [70, 84], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.48, 0.5],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.49, 0.49],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.5, 0.48],
  'value_type': 'float'},
 {'name': 'x_p1',
  'type': '

In [9]:
parameter_constraints

['x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_p1 - x_p1_1 >= 0.1',
 'x_p1_1 - x_p1_2 >= 0.1',
 'x_p2 - x_p2_1 >= 0.1',
 'x_p2_1 - x_p2_2 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1']

### Run tests to solve for best possible score    

In [10]:
parameterization, trial_index = Tag_client.get_next_trial()

[INFO 04-05 13:44:57] ax.service.ax_client: Generated new trial 0 with parameters {'z_sh': 12, 'z_sh_1': 13, 'z_sh_2': 13, 'z_p2': 30, 'z_p2_1': 32, 'z_p2_2': 28, 'z_r2': 86, 'z_r2_1': 74, 'z_r2_2': 74, 'x_s': 0.40634, 'x_s_1': 0.202612, 'x_s_2': -0.039872, 'x_p1': 0.860567, 'x_p1_1': 0.05466, 'x_p1_2': -0.905615, 'x_p2': 0.445594, 'x_p2_1': 0.329129, 'x_p2_2': -0.405654, 'x_r2': 1.007899, 'x_r2_1': -0.622691, 'x_r2_2': -0.861944} using model Sobol.


In [11]:
parameterization

{'z_sh': 12,
 'z_sh_1': 13,
 'z_sh_2': 13,
 'z_p2': 30,
 'z_p2_1': 32,
 'z_p2_2': 28,
 'z_r2': 86,
 'z_r2_1': 74,
 'z_r2_2': 74,
 'x_s': 0.4063396413438022,
 'x_s_1': 0.20261154625564815,
 'x_s_2': -0.03987187692895533,
 'x_p1': 0.8605672977305949,
 'x_p1_1': 0.05465959087014194,
 'x_p1_2': -0.9056153228506446,
 'x_p2': 0.44559385370463134,
 'x_p2_1': 0.3291290584392845,
 'x_p2_2': -0.4056536689773202,
 'x_r2': 1.0078993488475678,
 'x_r2_1': -0.6226905630342662,
 'x_r2_2': -0.8619438071921468}

In [12]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

21
z_sh: 12
z_sh_1: 13
z_sh_2: 13
z_p2: 30
z_p2_1: 32
z_p2_2: 28
z_r2: 86
z_r2_1: 74
z_r2_2: 74
x_s: 0.4063396413438022
x_s_1: 0.20261154625564815
x_s_2: -0.03987187692895533
x_p1: 0.8605672977305949
x_p1_1: 0.05465959087014194
x_p1_2: -0.9056153228506446
x_p2: 0.44559385370463134
x_p2_1: 0.3291290584392845
x_p2_2: -0.4056536689773202
x_r2: 1.0078993488475678
x_r2_1: -0.6226905630342662
x_r2_2: -0.8619438071921468


In [13]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 13:44:57] ax.service.ax_client: Completed trial 0 with data: {'Viable_Cross_Terms': (5, None)}.


NOTE - run the following through 'Exhausted' errors.

In [14]:
init_count = 0
while init_count < 42:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))
    except:
        print('Not this time')
    init_count = trial_index

[INFO 04-05 13:44:57] ax.service.ax_client: Generated new trial 1 with parameters {'z_sh': 9, 'z_sh_1': 12, 'z_sh_2': 6, 'z_p2': 28, 'z_p2_1': 30, 'z_p2_2': 32, 'z_r2': 80, 'z_r2_1': 85, 'z_r2_2': 74, 'x_s': 0.491751, 'x_s_1': 0.201916, 'x_s_2': 0.061762, 'x_p1': 0.280179, 'x_p1_1': -0.216793, 'x_p1_2': -0.674284, 'x_p2': 0.482764, 'x_p2_1': 0.303063, 'x_p2_2': -0.240026, 'x_r2': 1.027452, 'x_r2_1': -0.226139, 'x_r2_2': -0.731557} using model Sobol.
[INFO 04-05 13:44:57] ax.service.ax_client: Completed trial 1 with data: {'Viable_Cross_Terms': (16, None)}.
[INFO 04-05 13:44:58] ax.service.ax_client: Generated new trial 2 with parameters {'z_sh': 14, 'z_sh_1': 14, 'z_sh_2': 10, 'z_p2': 28, 'z_p2_1': 30, 'z_p2_2': 30, 'z_r2': 73, 'z_r2_1': 77, 'z_r2_2': 75, 'x_s': 0.496107, 'x_s_1': 0.012512, 'x_s_2': -0.492182, 'x_p1': 0.533445, 'x_p1_1': 0.067106, 'x_p1_2': -0.092637, 'x_p2': 0.318919, 'x_p2_1': -0.178784, 'x_p2_2': -0.4208, 'x_r2': 0.4999, 'x_r2_1': 0.174703, 'x_r2_2': -0.101379} usin

Not this time


[INFO 04-05 13:44:58] ax.service.ax_client: Generated new trial 4 with parameters {'z_sh': 8, 'z_sh_1': 14, 'z_sh_2': 6, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 25, 'z_r2': 74, 'z_r2_1': 75, 'z_r2_2': 70, 'x_s': 0.388911, 'x_s_1': 0.176081, 'x_s_2': 0.056943, 'x_p1': 0.874702, 'x_p1_1': 0.114893, 'x_p1_2': -0.669673, 'x_p2': 0.148211, 'x_p2_1': 0.013402, 'x_p2_2': -0.09417, 'x_r2': 0.051563, 'x_r2_1': -0.066229, 'x_r2_2': -0.587413} using model Sobol.
[INFO 04-05 13:44:58] ax.service.ax_client: Completed trial 4 with data: {'Viable_Cross_Terms': (7, None)}.


Not this time


[INFO 04-05 13:44:59] ax.service.ax_client: Generated new trial 5 with parameters {'z_sh': 7, 'z_sh_1': 12, 'z_sh_2': 7, 'z_p2': 33, 'z_p2_1': 30, 'z_p2_2': 25, 'z_r2': 72, 'z_r2_1': 85, 'z_r2_2': 83, 'x_s': 0.367652, 'x_s_1': -0.042555, 'x_s_2': -0.17255, 'x_p1': 0.785026, 'x_p1_1': 0.661065, 'x_p1_2': 0.05197, 'x_p2': 0.247682, 'x_p2_1': 0.080662, 'x_p2_2': -0.165178, 'x_r2': 0.341651, 'x_r2_1': 0.13408, 'x_r2_2': -0.271007} using model Sobol.
[INFO 04-05 13:44:59] ax.service.ax_client: Completed trial 5 with data: {'Viable_Cross_Terms': (15, None)}.
[INFO 04-05 13:44:59] ax.service.ax_client: Generated new trial 6 with parameters {'z_sh': 7, 'z_sh_1': 12, 'z_sh_2': 11, 'z_p2': 30, 'z_p2_1': 33, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 71, 'z_r2_2': 77, 'x_s': 0.448106, 'x_s_1': -0.091749, 'x_s_2': -0.354004, 'x_p1': 0.514511, 'x_p1_1': -0.268157, 'x_p1_2': -0.64201, 'x_p2': 0.415432, 'x_p2_1': -0.214854, 'x_p2_2': -0.431319, 'x_r2': -0.580071, 'x_r2_1': -0.950087, 'x_r2_2': -1.181672} us

...

[INFO 04-05 13:45:08] ax.service.ax_client: Generated new trial 31 with parameters {'z_sh': 11, 'z_sh_1': 9, 'z_sh_2': 11, 'z_p2': 28, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 77, 'z_r2_1': 80, 'z_r2_2': 75, 'x_s': 0.304189, 'x_s_1': 0.147881, 'x_s_2': -0.2788, 'x_p1': 0.919202, 'x_p1_1': 0.609352, 'x_p1_2': -0.109904, 'x_p2': 0.356424, 'x_p2_1': -0.081116, 'x_p2_2': -0.439565, 'x_r2': -0.149384, 'x_r2_1': -0.615048, 'x_r2_2': -1.394528} using model Sobol.
[INFO 04-05 13:45:08] ax.service.ax_client: Completed trial 31 with data: {'Viable_Cross_Terms': (4, None)}.
[INFO 04-05 13:45:09] ax.service.ax_client: Generated new trial 32 with parameters {'z_sh': 7, 'z_sh_1': 7, 'z_sh_2': 10, 'z_p2': 29, 'z_p2_1': 29, 'z_p2_2': 26, 'z_r2': 76, 'z_r2_1': 77, 'z_r2_2': 74, 'x_s': 0.494681, 'x_s_1': -0.001794, 'x_s_2': -0.198794, 'x_p1': 0.520821, 'x_p1_1': -0.527788, 'x_p1_2': -0.843626, 'x_p2': 0.122679, 'x_p2_1': -0.179833, 'x_p2_2': -0.330934, 'x_r2': 1.226565, 'x_r2_1': 0.2284, 'x_r2_2': -1.495501}

Not this time


[INFO 04-05 13:45:09] ax.service.ax_client: Generated new trial 34 with parameters {'z_sh': 12, 'z_sh_1': 7, 'z_sh_2': 10, 'z_p2': 29, 'z_p2_1': 32, 'z_p2_2': 31, 'z_r2': 82, 'z_r2_1': 85, 'z_r2_2': 77, 'x_s': 0.468549, 'x_s_1': -0.143975, 'x_s_2': -0.334931, 'x_p1': 0.564765, 'x_p1_1': 0.24362, 'x_p1_2': -0.200269, 'x_p2': 0.422512, 'x_p2_1': 0.147548, 'x_p2_2': -0.494425, 'x_r2': -0.141912, 'x_r2_1': -0.355214, 'x_r2_2': -0.638365} using model Sobol.
[INFO 04-05 13:45:09] ax.service.ax_client: Completed trial 34 with data: {'Viable_Cross_Terms': (16, None)}.
[INFO 04-05 13:45:10] ax.service.ax_client: Generated new trial 35 with parameters {'z_sh': 12, 'z_sh_1': 8, 'z_sh_2': 5, 'z_p2': 35, 'z_p2_1': 31, 'z_p2_2': 25, 'z_r2': 78, 'z_r2_1': 71, 'z_r2_2': 81, 'x_s': 0.195113, 'x_s_1': -0.281582, 'x_s_2': -0.445514, 'x_p1': 0.722113, 'x_p1_1': 0.170112, 'x_p1_2': 0.038937, 'x_p2': 0.481426, 'x_p2_1': 0.251735, 'x_p2_2': -0.172327, 'x_r2': 0.818466, 'x_r2_1': -1.282179, 'x_r2_2': -1.44565

Not this time


[INFO 04-05 13:45:11] ax.service.ax_client: Generated new trial 40 with parameters {'z_sh': 8, 'z_sh_1': 13, 'z_sh_2': 9, 'z_p2': 30, 'z_p2_1': 27, 'z_p2_2': 33, 'z_r2': 72, 'z_r2_1': 76, 'z_r2_2': 73, 'x_s': 0.29969, 'x_s_1': -0.325354, 'x_s_2': -0.426923, 'x_p1': 0.206654, 'x_p1_1': 0.045408, 'x_p1_2': -0.6756, 'x_p2': 0.28375, 'x_p2_1': 0.143823, 'x_p2_2': -0.028566, 'x_r2': 1.434858, 'x_r2_1': 0.212313, 'x_r2_2': -0.470594} using model Sobol.
[INFO 04-05 13:45:11] ax.service.ax_client: Completed trial 40 with data: {'Viable_Cross_Terms': (15, None)}.
[INFO 04-05 13:45:11] ax.service.ax_client: Generated new trial 41 with parameters {'z_sh': 10, 'z_sh_1': 8, 'z_sh_2': 8, 'z_p2': 28, 'z_p2_1': 30, 'z_p2_2': 27, 'z_r2': 75, 'z_r2_1': 75, 'z_r2_2': 74, 'x_s': 0.434139, 'x_s_1': 0.156853, 'x_s_2': 0.017333, 'x_p1': 0.767957, 'x_p1_1': 0.233234, 'x_p1_2': -0.417394, 'x_p2': 0.477265, 'x_p2_1': 0.258775, 'x_p2_2': 0.156054, 'x_r2': 0.786017, 'x_r2_1': -0.884349, 'x_r2_2': -1.143639} using

In [15]:
import importlib
%load_ext autoreload

In [16]:
%autoreload 2
from utils import list_to_param

In [17]:
z_sh = np.array([10, 8, 6])
z_p1 = np.array([42, 40, 41])
z_p2 = np.array([32, 31, 30])
z_r2 = np.array([80, 78, 77])
x_s = np.array([0.5, 0.4, 0.3])
x_p1 = np.array([0.8, 0.7, 0.6])
x_p2 = np.array([0.4, 0.3, 0.2])
x_r2 = np.array([1.2, 1.0, 0.8])

Yi_vals = [z_sh, z_p2, z_r2, x_s, x_p1, x_p2, x_r2] #list of arrays
Yi_list = list_to_param(Yi_vals)

trial_deal = Tag_client.attach_trial(
        parameters=Yi_list
    )
Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(Yi_list)))

[INFO 04-05 13:45:48] ax.core.experiment: Attached custom parameterizations [{'z_sh': 10, 'z_sh_1': 8, 'z_sh_2': 6, 'z_p2': 32, 'z_p2_1': 31, 'z_p2_2': 30, 'z_r2': 80, 'z_r2_1': 78, 'z_r2_2': 77, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.8, 'x_p1_1': 0.7, 'x_p1_2': 0.6, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.2, 'x_r2_1': 1.0, 'x_r2_2': 0.8}] as trial 43.


[INFO 04-05 13:45:48] ax.service.ax_client: Completed trial 43 with data: {'Viable_Cross_Terms': (30, None)}.


In [37]:
best_so_far =  {'z_sh': 8, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 72, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.635967, 'x_p1_1': 0.535967, 'x_p1_2': 0.435967, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.072711, 'x_r2_1': 0.972711, 'x_r2_2': 0.872711}
b_guess = {'z_sh': 8, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 72, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.636, 'x_p1_1': 0.536, 'x_p1_2': 0.436, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.073, 'x_r2_1': 0.973, 'x_r2_2': 0.873}
c_guess = {'z_sh': 7, 'z_sh_1': 8, 'z_sh_2': 5, 'z_p2': 33, 'z_p2_1': 31, 'z_p2_2': 28, 'z_r2': 80, 'z_r2_1': 73, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.649934, 'x_p1_1': 0.549934, 'x_p1_2': 0.449934, 'x_p2': 0.452019, 'x_p2_1': 0.352019, 'x_p2_2': 0.252019, 'x_r2': 1.068522, 'x_r2_1': 0.968522, 'x_r2_2': 0.868522}
d_guess = {'z_sh': 7, 'z_sh_1': 8, 'z_sh_2': 5, 'z_p2': 32, 'z_p2_1': 31, 'z_p2_2': 25, 'z_r2': 80, 'z_r2_1': 82, 'z_r2_2': 81, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.618688, 'x_p1_1': 0.518688, 'x_p1_2': 0.418688, 'x_p2': 0.475209, 'x_p2_1': 0.375209, 'x_p2_2': 0.275209, 'x_r2': 1.070261, 'x_r2_1': 0.970261, 'x_r2_2': 0.870261}
new_guess = list_to_param((np.array((14, 12, 10))/2,
 (27, 30, 31),
 (77, 78, 80),
 (0.3, 0.4, 0.5)[::-1],
 (0.6, 0.7, 0.8)[::-1],
 (0.2, 0.3, 0.4)[::-1],
 (0.8, 0.9, 1)[::-1]))

trial_deal = Tag_client.attach_trial(
        parameters=new_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(new_guess)))

[INFO 04-05 19:23:12] ax.core.experiment: Attached custom parameterizations [{'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 27, 'z_p2_1': 30, 'z_p2_2': 31, 'z_r2': 77, 'z_r2_1': 78, 'z_r2_2': 80, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.8, 'x_p1_1': 0.7, 'x_p1_2': 0.6, 'x_p2': 0.4, 'x_p2_1': 0.3, 'x_p2_2': 0.2, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8}] as trial 632.
[INFO 04-05 19:23:12] ax.service.ax_client: Completed trial 632 with data: {'Viable_Cross_Terms': (79, None)}.


In [19]:
trial_deal = Tag_client.attach_trial(
        parameters=best_so_far
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(best_so_far)))

trial_deal = Tag_client.attach_trial(
        parameters=b_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(b_guess)))

trial_deal = Tag_client.attach_trial(
        parameters=c_guess
    )

Tag_client.complete_trial(trial_index=trial_deal[-1], raw_data=score_vals(param_to_list(c_guess)))

[INFO 04-05 13:45:48] ax.core.experiment: Attached custom parameterizations [{'z_sh': 8, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 72, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.635967, 'x_p1_1': 0.535967, 'x_p1_2': 0.435967, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.072711, 'x_r2_1': 0.972711, 'x_r2_2': 0.872711}] as trial 45.
[INFO 04-05 13:45:48] ax.service.ax_client: Completed trial 45 with data: {'Viable_Cross_Terms': (90, None)}.
[INFO 04-05 13:45:48] ax.core.experiment: Attached custom parameterizations [{'z_sh': 8, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 29, 'z_p2_1': 27, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 72, 'z_r2_2': 71, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.636, 'x_p1_1': 0.536, 'x_p1_2': 0.436, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.073, 'x_r2_1': 0.973, 'x_r2_2': 0.873}] as trial 46.


[INFO 04-05 13:45:48] ax.service.ax_client: Completed trial 46 with data: {'Viable_Cross_Terms': (90, None)}.


In [20]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Viable_Cross_Terms,z_sh,z_sh_1,z_sh_2,z_p2,...,x_s_2,x_p1,x_p1_1,x_p1_2,x_p2,x_p2_1,x_p2_2,x_r2,x_r2_1,x_r2_2
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,5.0,12,13,13,30,...,-0.039872,0.860567,0.054660,-0.905615,0.445594,0.329129,-0.405654,1.007899,-0.622691,-0.861944
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,16.0,9,12,6,28,...,0.061762,0.280179,-0.216793,-0.674284,0.482764,0.303063,-0.240026,1.027452,-0.226139,-0.731557
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,12.0,14,14,10,28,...,-0.492182,0.533445,0.067106,-0.092637,0.318919,-0.178784,-0.420800,0.499900,0.174703,-0.101379
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,20.0,14,9,7,31,...,-0.387237,0.271144,-0.010371,-0.357360,0.408685,0.247238,-0.078927,1.035394,-0.490845,-1.496787
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,7.0,8,14,6,29,...,0.056943,0.874702,0.114893,-0.669673,0.148211,0.013402,-0.094170,0.051563,-0.066229,-0.587413
5,5,5_0,COMPLETED,Sobol,GenerationStep_0,15.0,7,12,7,33,...,-0.172550,0.785026,0.661065,0.051970,0.247682,0.080662,-0.165178,0.341651,0.134080,-0.271007
6,6,6_0,COMPLETED,Sobol,GenerationStep_0,5.0,7,12,11,30,...,-0.354004,0.514511,-0.268157,-0.642010,0.415432,-0.214854,-0.431319,-0.580071,-0.950087,-1.181672
7,7,7_0,COMPLETED,Sobol,GenerationStep_0,11.0,12,6,8,35,...,-0.382414,0.852140,0.151129,-0.352273,0.221513,0.046887,-0.239381,0.160039,-0.709710,-1.358825
8,8,8_0,COMPLETED,Sobol,GenerationStep_0,6.0,11,7,11,31,...,-0.468167,0.352667,0.021823,-0.496864,0.095443,-0.056417,-0.237519,0.375028,-1.225837,-1.444941
9,9,9_0,COMPLETED,Sobol,GenerationStep_0,11.0,13,9,9,30,...,-0.076067,-0.319844,-0.573911,-0.965283,0.128872,-0.025865,-0.433866,0.907326,0.662571,-1.392847


In [21]:
render(Tag_client.get_optimization_trace())

In [22]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 13:46:08] ax.service.ax_client: Generated new trial 47 with parameters {'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 27, 'z_p2_1': 26, 'z_p2_2': 33, 'z_r2': 78, 'z_r2_1': 71, 'z_r2_2': 70, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.659814, 'x_p1_1': 0.559814, 'x_p1_2': 0.459814, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.458836, 'x_r2_1': 1.358836, 'x_r2_2': 1.258836} using model BoTorch.
[INFO 04-05 13:46:08] ax.service.ax_client: Completed trial 47 with data: {'Viable_Cross_Terms': (0, None)}.
[INFO 04-05 13:46:26] ax.service.ax_client: Generated new trial 48 with parameters {'z_sh': 15, 'z_sh_1': 11, 'z_sh_2': 5, 'z_p2': 35, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 86, 'z_r2_1': 71, 'z_r2_2': 70, 'x_s': 0.5, 'x_s_1': -0.4, 'x_s_2': -0.5, 'x_p1': 1.0, 'x_p1_1': 0.9, 'x_p1_2': 0.8, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': -0.102365, 'x_r2': 1.162948, 'x_r2_1': -1.4, 'x_r2_2': -1.5} using model BoTorch.
[INFO 04-05 13:46:26] ax.service.ax_client: Completed trial 48

In [46]:
Tag_client.generation_strategy.trials_as_df.iloc[1004]

trial_index                       1004
arm_name                        1004_0
trial_status                 COMPLETED
generation_method              BoTorch
generation_node       GenerationStep_1
Viable_Cross_Terms               105.0
z_sh                                 7
z_sh_1                               8
z_sh_2                               5
z_p2                                29
z_p2_1                              32
z_p2_2                              31
z_r2                                80
z_r2_1                              82
z_r2_2                              83
x_s                                0.5
x_s_1                              0.4
x_s_2                              0.3
x_p1                          0.613076
x_p1_1                        0.513076
x_p1_2                        0.413076
x_p2                               0.5
x_p2_1                             0.4
x_p2_2                             0.3
x_r2                          1.077693
x_r2_1                   

In [24]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")
print(score_vals(param_to_list(best_parameters)))

z_sh: 12
z_sh_1: 8
z_sh_2: 5
z_p2: 35
z_p2_1: 31
z_p2_2: 25
z_r2: 78
z_r2_1: 71
z_r2_2: 81
x_s: 0.19511320931836962
x_s_1: -0.28158239942044017
x_s_2: -0.44551438799127935
x_p1: 0.7221133673936129
x_p1_1: 0.17011185156181452
x_p1_2: 0.038936774730682355
x_p2: 0.4814263208396733
x_p2_1: 0.2517351661995053
x_p2_2: -0.17232676306739453
x_r2: 0.8184660379588604
x_r2_1: -1.2821789846755565
x_r2_2: -1.445656919553876
23


In [25]:
param_to_list(best_parameters)

[array([12,  8,  5]),
 array([35, 31, 25]),
 array([78, 71, 81]),
 array([ 0.19511321, -0.2815824 , -0.44551439]),
 array([0.72211337, 0.17011185, 0.03893677]),
 array([ 0.48142632,  0.25173517, -0.17232676]),
 array([ 0.81846604, -1.28217898, -1.44565692])]

In [26]:
render(Tag_client.get_optimization_trace())

In [27]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 14:15:58] ax.service.ax_client: Generated new trial 147 with parameters {'z_sh': 8, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 33, 'z_p2_1': 31, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 85, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.629718, 'x_p1_1': 0.529718, 'x_p1_2': 0.429718, 'x_p2': 0.410605, 'x_p2_1': 0.310605, 'x_p2_2': 0.210605, 'x_r2': 0.981954, 'x_r2_1': 0.881954, 'x_r2_2': 0.781954} using model BoTorch.
[INFO 04-05 14:15:58] ax.service.ax_client: Completed trial 147 with data: {'Viable_Cross_Terms': (85, None)}.
[INFO 04-05 14:16:17] ax.service.ax_client: Generated new trial 148 with parameters {'z_sh': 12, 'z_sh_1': 12, 'z_sh_2': 6, 'z_p2': 34, 'z_p2_1': 30, 'z_p2_2': 25, 'z_r2': 86, 'z_r2_1': 76, 'z_r2_2': 83, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.869118, 'x_p1_1': 0.769118, 'x_p1_2': 0.669118, 'x_p2': 0.390927, 'x_p2_1': 0.290927, 'x_p2_2': 0.190927, 'x_r2': 0.794506, 'x_r2_1': 0.694506, 'x_r2_2': 0.594506} using model BoTorch.
[INFO 04-05 

In [44]:
render(Tag_client.get_optimization_trace())

In [ ]:
#Viable_Cross_Terms               133.0
z_sh = [ 7, 8, 9]
z_p2 = [28,31,26]
z_r2 = [82,74,83]
x_s = [ 0.5, 0.4, 0.3]
x_p1 = [ 1.0, 0.9, 0.8]
x_p2 = [-0.3,-0.4,-0.5]
x_r2 = [ 0.461185, 0.361185, 0.261185]

In [42]:
for i in range(800):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-06 08:44:12] ax.modelbridge.base: Leaving out out-of-design observations for arms: 729_0, 977_0, 489_0, 686_0, 775_0, 899_0
[INFO 04-06 08:46:47] ax.service.ax_client: Generated new trial 1029 with parameters {'z_sh': 15, 'z_sh_1': 11, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 79, 'z_r2_1': 81, 'z_r2_2': 84, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.643009, 'x_p1_1': 0.543009, 'x_p1_2': 0.443009, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.5, 'x_r2_1': -0.9846, 'x_r2_2': -1.5} using model BoTorch.
[INFO 04-06 08:46:47] ax.service.ax_client: Completed trial 1029 with data: {'Viable_Cross_Terms': (0, None)}.
[INFO 04-06 08:46:47] ax.modelbridge.base: Leaving out out-of-design observations for arms: 729_0, 977_0, 489_0, 686_0, 775_0, 899_0
[INFO 04-06 08:51:00] ax.service.ax_client: Generated new trial 1030 with parameters {'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 35, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 80, 'z_r2_1': 85, 'z_r2_2': 84, 'x_s'

KeyboardInterrupt: 